<a href="https://colab.research.google.com/github/mvassche/colab/blob/main/colab_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install transformers datasets
from transformers import pipeline
classifier = pipeline("sentiment-analysis")
#classifier("We are very happy to show you the 🤗 Transformers library.")
classifier("Je suis très dissu.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.9970002770423889}]

In [5]:
import torch
from transformers import pipeline

speech_recognizer = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from datasets import load_dataset, Audio
dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")
dataset = dataset.cast_column("audio", Audio(sampling_rate=speech_recognizer.feature_extractor.sampling_rate))

In [12]:
result = speech_recognizer(dataset[:4]["audio"])
print([d["text"] for d in result])

['I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER HOW DO I PROCEED WITH DOING THAT', "FONDERING HOW I'D SET UP A JOIN TO HELL T WITH MY WIFE AND WHERE THE AP MIGHT BE", "I I'D LIKE TOY SET UP A JOINT ACCOUNT WITH MY PARTNER I'M NOT SEEING THE OPTION TO DO IT ON THE APSO I CALLED IN TO GET SOME HELP CAN I JUST DO IT OVER THE PHONE WITH YOU AND GIVE YOU THE INFORMATION OR SHOULD I DO IT IN THE AP AN I'M MISSING SOMETHING UQUETTE HAD PREFERRED TO JUST DO IT OVER THE PHONE OF POSSIBLE THINGS", 'HOW DO I FURN A JOINA COUT']


In [15]:
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
import datasets

dataset = datasets.load_dataset("imdb", name="plain_text", split="unsupervised")
pipe = pipeline("text-classification")
for out in pipe(KeyDataset(dataset, "text"), batch_size=8, truncation="only_first"):
    print(out)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'label': 'POSITIVE', 'score': 0.5735197067260742}
{'label': 'POSITIVE', 'score': 0.9997995495796204}
{'label': 'POSITIVE', 'score': 0.9996894598007202}
{'label': 'NEGATIVE', 'score': 0.9103306531906128}
{'label': 'NEGATIVE', 'score': 0.9979425072669983}
{'label': 'NEGATIVE', 'score': 0.9977090358734131}
{'label': 'POSITIVE', 'score': 0.9988546371459961}
{'label': 'POSITIVE', 'score': 0.9998350143432617}
{'label': 'POSITIVE', 'score': 0.986367404460907}
{'label': 'POSITIVE', 'score': 0.9990979433059692}
{'label': 'NEGATIVE', 'score': 0.9988611936569214}
{'label': 'NEGATIVE', 'score': 0.7546901106834412}
{'label': 'POSITIVE', 'score': 0.9954828023910522}
{'label': 'POSITIVE', 'score': 0.9992594122886658}
{'label': 'POSITIVE', 'score': 0.9948545694351196}
{'label': 'NEGATIVE', 'score': 0.9996827840805054}
{'label': 'POSITIVE', 'score': 0.999333918094635}
{'label': 'NEGATIVE', 'score': 0.7249546051025391}
{'label': 'POSITIVE', 'score': 0.8955298662185669}
{'label': 'POSITIVE', 'score': 0.

KeyboardInterrupt: ignored